In [35]:
import numpy as np
import cmath
try:
    import scipy
except: 
    !pip install scipy
from scipy.integrate import quad

In [36]:
def complex_quadrature(func, lower, upper, **kwargs):
    def real_func(x):
        return np.real(func(x))
    def imag_func(x):
        return np.imag(func(x))
    real_integral = quad(real_func, lower, upper, **kwargs)
    imag_integral = quad(imag_func, lower, upper, **kwargs)
    return real_integral[0] + 1j*imag_integral[0]

In [54]:
class Wielomian:
    def __init__(self):
        self.lista_pierwiastkow = []
        self.wspolczynniki = []
        self.stopien = None
        self.liczba_prob = 0
        
    def load_polynomial(self, plik):
        with open(plik, 'r') as f:
            lines = f.read()  

        arr = [float(l) for l in lines.splitlines()]
        self.wspolczynniki = arr[1:]
        self.stopien = int(arr[0])
        #return wspolczynniki
    
    
    def compute_poly(self, x):
        fun = 0
        if len(self.wspolczynniki) < self.stopien + 1:
            print(f'Podano za mało współczynników, potrzeba ich {self.stopien+1}. Popraw w pliku.')
            return 0

        if len(self.wspolczynniki) > self.stopien + 1:
            print(f'Podano więcej współczynników niż oczekiwana liczba: {self.stopien+1}. Popraw w pliku.')
            return 0

        for i in range(self.stopien+1):
            fun += self.wspolczynniki[i] * np.power(x, i)
            
        return fun
    
    
    def compute_deriv_poly(self, x):
        fun_deriv = 0
        if len(self.wspolczynniki) < self.stopien + 1:
            print(f'Podano za mało współczynników, potrzeba ich {self.stopien+1}. Popraw w pliku.')
            return 0

        if len(self.wspolczynniki) > self.stopien + 1:
            print(f'Podano więcej współczynników niż oczekiwana liczba: {self.stopien+1}. Popraw w pliku.')
            return 0

        for i in range(1, self.stopien+1):
            fun_deriv += i * self.wspolczynniki[i] * np.power(x, i-1)
            
        return fun_deriv
    
    
    def residuum(self, R, eps=1e-6):

        def p(x):
            return np.sum([self.wspolczynniki[i]*np.power(x,i) for i in range(self.stopien+1)])

        def p_prime(x):
            return np.sum([i*self.wspolczynniki[i]*np.power(x,i-1) for i in range(1, self.stopien+1)])

        def P(x):
            try:
                return p_prime(x)/p(x)
            except:
                return 0

        def gamma(x):
            return R*np.exp(1j*x) # R*e^{it}

        def gamma_prime(x):
            return R*1j*np.exp(1j*x) # R*i*e^{it}

        if p(R) == 0:
            print("Jeden z pierwiastków wielomianu znajduje się na konturze obszaru")
            result = int(np.floor(np.real(complex_quadrature(lambda x: 
                    P(gamma(x))*gamma_prime(x)/(2*np.pi*1j), 0, 2*np.pi)))) + 1 # + krotnosc_pierw

        else: result = np.real(complex_quadrature(lambda x: P(gamma(x))*gamma_prime(x)/(2*np.pi*1j), 0, 2*np.pi))

        return result
    
    
    def Newton(self, n_steps=2e5, eps=1e-8):

        k = 1
        exit = False
        startowy = False

        while(startowy==False):
            try:
                z0 = complex(input("Wprowadź punkt startowy (rzeczywisty lub zespolony): "))
                startowy = True
            except:
                print("Punkt startowy musi być typu 'complex'. Spróbuj ponownie:")
                continue

        f0 = self.compute_poly(z0)
        z1 = z0 - 1

        while((k <= n_steps) and (np.abs(z0 - z1) > eps) and (np.abs(f0) > eps)):

            try:
                fp0 = self.compute_deriv_poly(z0)
                z1 = z0
                z0 = z0 - f0/fp0
            except:
                print("Pochodna w punkcie startowym wynosi 0!")
                pass

            if np.abs(fp0) < eps:
                print("Zły punkt startowy")
                exit = True
                break

            f0 = self.compute_poly(z0)
            k = k+1

        if exit:
            return 0

        if k > n_steps:
            print("Przekroczono limit kroków algorytmu, zły punkt startowy.")
            cont = input("Czy chcesz kontynuować? [T/N] ")
            if cont.lower() in ["n", "no", "nie"]: 
                print(f"Miejsca zerowe tego wielomianu to: {self.lista_pierwiastkow}.")
                return 0

            else:
                self.Newton(self.wspolczynniki, n_steps, eps)
        else:
            if len(self.lista_pierwiastkow) != 0:
                if np.round(z0, 8) in self.lista_pierwiastkow:
                    print("Znaleziony pierwiastek jest już na liście.")
                    cont = input("Czy chcesz kontynuować? [T/N] ")
                    if cont.lower() in ["n", "no", "nie"]: 
                        print(f"Miejsca zerowe tego wielomianu to: {self.lista_pierwiastkow}.")
                        return

                else: 
                    self.lista_pierwiastkow.append(np.round(z0, 8))

            else: 
                self.lista_pierwiastkow.append(np.round(z0, 8))

            if len(self.lista_pierwiastkow) == len(self.wspolczynniki)-1:
                print(f"Miejsca zerowe tego wielomianu to: {self.lista_pierwiastkow}.")
            else:
                self.Newton()

In [55]:
nowy_wielomian = Wielomian()
nowy_wielomian.load_polynomial('wielomian.txt')

print(nowy_wielomian.wspolczynniki)

print(nowy_wielomian.residuum(R=0.7))

#nowy_wielomian.Newton()

[0.0, 1.0, -2.0]
2.0
